In [ ]:
# First run Python to mount google drive (from: Runtime>>Change Runtime type>>Runtime type, select Python3)
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Install and import packages for Julia environment

The Runtime type should be changed to Julia

In [ ]:
using Pkg
Pkg.Registry.add(RegistrySpec(url="https://github.com/ACEsuit/ACEregistry"))
Pkg.add(Pkg.PackageSpec(;name="ACEpotentials", version="0.6.5"))
Pkg.Registry.add("General")
Pkg.add("JuLIP")
using ACEpotentials

# Tungsten (W) Dtat and fit model

## Load and split dataset

In [ ]:
path_to_xyz_file = "random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250_withoutLatt3.188.xyz"

"/content/drive/MyDrive/Colab_files/Thesis/MACE/Generate Synthetic structures MACE/Curvature Snapshots/Mo_random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250.xyz"

In [ ]:
w_data_xyz = read_extxyz(path_to_xyz_file)
length(w_data_xyz)

176

In [ ]:
println(w_data_xyz[3].data)

Dict{Any, JuLIP.JData{Float64}}("energy_per_atom" => JuLIP.JData{Float64}(Inf, 0.0, -10.44579766117498), "forces" => JuLIP.JData{Float64}(Inf, 0.0, StaticArraysCore.SVector{3, Float64}[[0.0372479, 0.141572, -0.12500411], [-0.03724789, -0.14157191, 0.12500434]]), "sign" => JuLIP.JData{Float64}(Inf, 0.0, 1), "probe_id" => JuLIP.JData{Float64}(Inf, 0.0, 1), "energy" => JuLIP.JData{Float64}(Inf, 0.0, -20.888731002807617), "origin" => JuLIP.JData{Float64}(Inf, 0.0, "W_base0"))


In [ ]:
println(w_data_xyz[1].data.keys)

Any["energy_per_atom", #undef, "forces", #undef, "sign", #undef, #undef, "probe_id", #undef, #undef, "energy", #undef, #undef, #undef, #undef, "origin"]


In [ ]:
w_data_train_xyz = w_data_xyz[1:2:end]
println(w_data_train_xyz)
println(length(w_data_train_xyz))


Atoms[JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.

In [ ]:
w_data_test_xyz = w_data_xyz[2:2:end]
println(w_data_test_xyz)
println(length(w_data_test_xyz))

Atoms[JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.

## Load already splited data

In [ ]:
path_to_xyz_file_train = "/content/drive/MyDrive/Colab_files/Thesis/MACE/MACE MD trajectories/MD trajectories (Alice)/Tungsten/BCC/md_tungsten_bulk_N16_steps1500000_interval500_Tmix_W_3.188_bcc_training_data.xyz"

path_to_xyz_file_test = "/content/drive/MyDrive/Colab_files/Thesis/MACE/MACE MD trajectories/MD trajectories (Alice)/Tungsten/BCC/md_tungsten_bulk_N16_steps1500000_interval500_Tmix_W_3.188_bcc_test_data.xyz"

path_to_xyz_file_validation = "/content/drive/MyDrive/Colab_files/Thesis/MACE/MACE MD trajectories/MD trajectories (Alice)/Tungsten/BCC/md_tungsten_bulk_N16_steps1500000_interval500_Tmix_W_3.188_bcc_validation_data.xyz"

w_data_train_xyz = read_extxyz(path_to_xyz_file_train)
w_data_test_xyz = read_extxyz(path_to_xyz_file_test)
w_data_validation_xyz = read_extxyz(path_to_xyz_file_validation);

LoadError: UndefVarError: `read_extxyz` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## Fit single model

In [ ]:
# This controls the models hyperparamets
order = 4
totaldegree = 9
rcut = 4.5
hyperparams = (elements = [:W],
					order = order,
					totaldegree = totaldegree,
					rcut = rcut,
					Eref = [:W => -5.07176726]) # Change this line to the isolated atom value of the foundation model or method # from MACE MP L1 = -5.07176726, from DFT  = -4.5711108, from MACE-MP0b3_A13_MD_ft = -4.65658356,
model = acemodel(; hyperparams...)
@show length(model.basis);


weights = Dict(
        "W" => Dict("E" => 1.0, "F" => 1.0 , "V" => 1.0 ));

# The solver can be changed too

#solver = ACEfit.LSQR()

solver = ACEfit.RRQR()

#solver = ACEfit.BLR();
#ACEfit.BLR(Dict{Any, Any}());
data_keys = (energy_key = "energy", force_key = "forces");

length(model.basis) = 104


In [ ]:
# This is solving for the parameters of the linear model. Test for XYZ file

result = acefit!( model, w_data_train_xyz, solver=solver, weights=weights; data_keys...);

┌─────────┬──────────┬───────┬─────┬───────┬──────┐
│    Type │ #Configs │ #Envs │  #E │    #F │   #V │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│ default │      540 │  8640 │ 540 │ 25920 │    0 │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│   total │      540 │  8640 │ 540 │ 25920 │    0 │
│ missing │        0 │     0 │   0 │     0 │ 3240 │
└─────────┴──────────┴───────┴─────┴───────┴──────┘


[ Info: Assembling linear problem.
[ Info:   - Creating feature matrix with size (26460, 104).
[ Info:   - Beginning assembly with processor count:  1.
Progress: 100%|█████████████████████████████████████████| Time: 0:06:52
[ Info:   - Assembly completed.
[ Info: Assembling full weight vector.
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [ ]:

@info("Training Errors")
train_res = ACEpotentials.linear_errors(w_data_train_xyz, model; data_keys...);

@info("Test Errors")
test_res = ACEpotentials.linear_errors(w_data_test_xyz, model; data_keys...);

@info("Validation Errors")
validation_res = ACEpotentials.linear_errors(w_data_validation_xyz, model; data_keys...);

[ Info: Training Errors


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.318 │    0.016 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.318 │    0.016 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.225 │    0.011 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.225 │    0.011 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: MAE Table
[ Info: Test Errors


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.375 │    0.018 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.375 │    0.018 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.239 │    0.012 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.239 │    0.012 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table
[ Info: Validation Errors


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.286 │    0.015 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.286 │    0.015 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   0.192 │    0.011 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   0.192 │    0.011 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table


In [ ]:
println(train_res["rmse"]["set"]["E"]*1000)
println(train_res["rmse"]["set"]["F"])
println(train_res["mae"]["set"]["E"]*1000)
println(train_res["mae"]["set"]["F"])

0.31788023842356156
0.015678666342297115
0.22495121342133367
0.011188183441686818


In [ ]:
ACEpotentials.save_potential("W_Dataset_order$(order)_totaldegree$(totaldegree)_rcut$(rcut).json", model)

In [ ]:
import JuLIP: Atoms, energy, forces, mat

train_data_new_format = [ AtomsData(at; energy_key = "energy", force_key="forces",
                              v_ref = model.Vref)
                for at in w_data_train_xyz ] ;

test_data_new_format = [ AtomsData(at; energy_key = "energy", force_key="forces",
                              v_ref = model.Vref)
                for at in w_data_test_xyz ] ;

In [ ]:
model_rl = ACEpotentials.load_potential("W_Dataset_Tmix_order4_totaldegree9_rcut5.5.json", new_format=false)
forces1 = forces(model_rl, test_data_new_format[1].atoms)
energy(model_rl, test_data_new_format[1].atoms)

In [ ]:
# Energy
energy(model.potential,train_data_new_format[1].atoms)

-203.3288325749877

In [ ]:
# Force
forces(model.potential,train_data_new_format[1].atoms)

16-element Vector{StaticArraysCore.SVector{3, Float64}}:
 [-0.22951952029285394, -1.4613827628348588, 0.29100905596401105]
 [-1.114183045153922, 0.5894487554800207, 0.04363329162862328]
 [-0.09356670168870096, 0.3133097487542216, -0.09079347567172391]
 [0.4410584489362037, 0.15816767272380616, -0.4122429034300552]
 [1.197772047420301, 0.5913806512901658, -0.0330707940219952]
 [-0.2705348977588926, 0.28252209028112674, -0.17599476087983135]
 [0.10895137505303865, 0.3845973876449307, -0.5522498093175585]
 [-0.6244715485407539, 0.34206500289497654, 0.5603080598041572]
 [0.44447543523667576, -0.6480244437981071, 0.3128879264821265]
 [-0.020489866234981946, 0.05715802456959551, 0.5436427307640628]
 [0.35603118916697696, -1.0487887209178268, 0.1323339385258535]
 [-0.4797515161810111, 0.2887002967231851, -0.4875071262506907]
 [-0.43933686753942425, 0.013749713889012583, -0.2892030618014019]
 [-0.2149814732137827, 0.7841966060043433, 0.149818662038669]
 [0.9855021460318198, -0.3306673842166105

In [ ]:
# Time test for forces
t1 = time();

N_comp = 1000 # Number of repeats

for i in 1:N_comp
  forces(model_rl,train_data_new_format[1].atoms);
end

elapsed_time = time() - t1;

print("Time per force computation in ms: ")
print(elapsed_time/N_comp * 1000)
print(" ms \n")


t1 = time();
for i in 1:N_comp
  energy(model_rl,train_data_new_format[1].atoms);
end
elapsed_time = time() - t1;

print("Time per energy computation in ms: ")
print(elapsed_time/N_comp * 1000)
print(" ms \n")

## Fit ACE models using nest loop on Order and Total degree (Chapter 3)

In [ ]:
println(length(w_data_train_xyz))
println(length(w_data_test_xyz))
println(length(w_data_validation_xyz))

540
150
60


In [ ]:
order_list = [3, 4, 5, 6];
totaldegree_list = [3, 12] # It contains the range of the totaldegree variable, [Start, End]
rcut_list = [4.5, 7.5, 10.5];

In [ ]:

weights = Dict(
        "W" => Dict("E" => 1.0, "F" => 1.0 , "V" => 1.0 ));

# The solver can be changed too

solver = ACEfit.RRQR()
"""
solver = ACEfit.BLR()
ACEfit.BLR(Dict{Any, Any}())
"""
data_keys = (energy_key = "energy", force_key = "forces");

N_comp = 1000; # Number of repeats
dataset_temperature = "mix"; # 500, 800, 1200 or mix

In [ ]:
Pkg.add("Statistics")

   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [10745b16] + Statistics v1.11.1
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
using Statistics

In [ ]:
using Pkg
Pkg.add("DataFrames")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
using Pkg
Pkg.add("CSV")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
using DataFrames, CSV
results_df = DataFrame(order=Int[], totaldegree=Int[], rcut=Float64[], basis_lenght=Int[],
                       train_rmse_E=Float64[], train_rmse_F=Float64[],
                       train_mae_E=Float64[], train_mae_F=Float64[],
                       test_rmse_E=Float64[], test_rmse_F=Float64[],
                       test_mae_E=Float64[], test_mae_F=Float64[],
                       validation_rmse_E=Float64[], validation_rmse_F=Float64[],
                       validation_mae_E=Float64[], validation_mae_F=Float64[],
                       time_per_force_ms=Float64[], time_per_energy_ms=Float64[], time_for_train=Float64[])

Row,order,totaldegree,rcut,basis_lenght,train_rmse_E,train_rmse_F,train_mae_E,train_mae_F,test_rmse_E,test_rmse_F,test_mae_E,test_mae_F,validation_rmse_E,validation_rmse_F,validation_mae_E,validation_mae_F,time_per_force_ms,time_per_energy_ms,time_for_train
,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [ ]:
for order in order_list
    for totaldegree in totaldegree_list[1]:3:totaldegree_list[2]
        for rcut in rcut_list
            println("=============== order: $(order), totaldegree: $(totaldegree), rcut: $(rcut) ===============")

            try
                time_start_train = time()

                hyperparams = (elements = [:W],
                            order = order,
                            totaldegree = totaldegree,
                            rcut = rcut,
                            Eref = [:W => -5.07176726])
                model = acemodel(; hyperparams...)
                println("Basis' length: $(length(model.basis))")

                basis_length = length(model.basis)

                result = acefit!(model, w_data_train_xyz, solver=solver, weights=weights; data_keys...)

                time_end_train = time()
                time_for_train = time_end_train - time_start_train

                # Save model
                ACEpotentials.save_potential("W_Dataset_T$(dataset_temperature)_order$(order)_totaldegree$(totaldegree)_rcut$(rcut).json", model)

                # Error calculations
                train_res = ACEpotentials.linear_errors(w_data_train_xyz, model; data_keys...)
                test_res = ACEpotentials.linear_errors(w_data_test_xyz, model; data_keys...)
                validation_res = ACEpotentials.linear_errors(w_data_validation_xyz, model; data_keys...)

                train_rmse_E = train_res["rmse"]["set"]["E"] * 1000
                train_rmse_F = train_res["rmse"]["set"]["F"]
                train_mae_E  = train_res["mae"]["set"]["E"] * 1000
                train_mae_F  = train_res["mae"]["set"]["F"]

                test_rmse_E = test_res["rmse"]["set"]["E"] * 1000
                test_rmse_F = test_res["rmse"]["set"]["F"]
                test_mae_E  = test_res["mae"]["set"]["E"] * 1000
                test_mae_F  = test_res["mae"]["set"]["F"]

                validation_rmse_E = validation_res["rmse"]["set"]["E"] * 1000
                validation_rmse_F = validation_res["rmse"]["set"]["F"]
                validation_mae_E  = validation_res["mae"]["set"]["E"] * 1000
                validation_mae_F  = validation_res["mae"]["set"]["F"]

                println("RMSE: $(train_rmse_E) meV, $(train_rmse_F) eV/Å")
                println("MAE:  $(train_mae_E) meV, $(train_mae_F) eV/Å")

                # Force timing
                forces_runtime = []
                for j in 1:5
                    t1 = time()
                    for i in 1:N_comp
                        forces(model.potential, train_data_new_format[1].atoms)
                    end
                    push!(forces_runtime, time() - t1)
                end
                elapsed_time_forces = median(forces_runtime)
                println("Time per force computation in ms: $(elapsed_time_forces / N_comp * 1000) ms")

                # Energy timing
                energy_runtime = []
                for j in 1:5
                    t2 = time()
                    for i in 1:N_comp
                        energy(model.potential, train_data_new_format[1].atoms)
                    end
                    push!(energy_runtime, time() - t2)
                end
                elapsed_time_energy = median(energy_runtime)
                println("Time per energy computation in ms: $(elapsed_time_energy / N_comp * 1000) ms")

                # Save results
                push!(results_df, (order, totaldegree, rcut, basis_length,
                                train_rmse_E, train_rmse_F,
                                train_mae_E, train_mae_F,
                                test_rmse_E, test_rmse_F,
                                test_mae_E, test_mae_F,
                                validation_rmse_E, validation_rmse_F,
                                validation_mae_E, validation_mae_F,
                                elapsed_time_forces / N_comp * 1000,
                                elapsed_time_energy / N_comp * 1000,
                                time_for_train))

            catch e
                println("An error occurred: ", e)
                continue
            end
        end
    end
end

CSV.write("results$(dataset_temperature)_RRQR.csv", results_df)


## Fit ACE models using nest loop on Order and Total degree (Chapter 5)

In [ ]:
println(length(w_data_train_xyz))

88


In [ ]:
order_list = [3, 4, 5, 6];
totaldegree_list = [3, 12] # It contains the range of the totaldegree variable, [Start, End]
rcut_list = [4.5, 7.5, 10.5];

In [ ]:

weights = Dict(
        "W" => Dict("E" => 1.0, "F" => 1.0 , "V" => 1.0 ));

# The solver can be changed too

solver = ACEfit.RRQR()
"""
solver = ACEfit.BLR()
ACEfit.BLR(Dict{Any, Any}())
"""
data_keys = (energy_key = "energy", force_key = "forces");

N_comp = 1000; # Number of repeats for speed test
dataset_name, ext = splitext(basename(path_to_xyz_file)) # To use for saving model with same name


("random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250_withoutLatt3.188", ".xyz")

In [ ]:
using Pkg
Pkg.add("DataFrames")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
using Pkg
Pkg.add("CSV")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
using DataFrames, CSV
results_df = DataFrame(order=Int[], totaldegree=Int[], rcut=Float64[], basis_lenght=Int[],
                       train_rmse_E=Float64[], train_rmse_F=Float64[],
                       train_mae_E=Float64[], train_mae_F=Float64[],
                       time_for_train=Float64[])

Row,order,totaldegree,rcut,basis_lenght,train_rmse_E,train_rmse_F,train_mae_E,train_mae_F,time_for_train
,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64


In [ ]:
for order in order_list
    for totaldegree in totaldegree_list[1]:3:totaldegree_list[2]
        for rcut in rcut_list
            println("=============== order: $(order), totaldegree: $(totaldegree), rcut: $(rcut) ===============")

            try
                time_start_train = time()

                hyperparams = (elements = [:W],
                            order = order,
                            totaldegree = totaldegree,
                            rcut = rcut,
                            Eref = [:W => -5.07176726])
                model = acemodel(; hyperparams...)
                println("Basis' length: $(length(model.basis))")

                basis_length = length(model.basis)

                result = acefit!(model, w_data_train_xyz, solver=solver, weights=weights; data_keys...)

                time_end_train = time()
                time_for_train = time_end_train - time_start_train

                # Save model
                ACEpotentials.save_potential("m2_delta0.4_88strs/RRQR solver/W_$(dataset_name)_88strs_order$(order)_totaldegree$(totaldegree)_rcut$(rcut).json", model)

                # Error calculations
                train_res = ACEpotentials.linear_errors(w_data_train_xyz, model; data_keys...)

                train_rmse_E = train_res["rmse"]["set"]["E"] * 1000
                train_rmse_F = train_res["rmse"]["set"]["F"]
                train_mae_E  = train_res["mae"]["set"]["E"] * 1000
                train_mae_F  = train_res["mae"]["set"]["F"]


                println("RMSE: $(train_rmse_E) meV, $(train_rmse_F) eV/Å")
                println("MAE:  $(train_mae_E) meV, $(train_mae_F) eV/Å")

                # Save results
                push!(results_df, (order, totaldegree, rcut, basis_length,
                                train_rmse_E, train_rmse_F,
                                train_mae_E, train_mae_F,
                                time_for_train))

            catch e
                println("An error occurred: ", e)
                continue
            end
        end
    end
end

CSV.write("results$(dataset_name)_88strs_RRQR.csv", results_df)


# Energy and Force Timing

In [ ]:
Pkg.add("Statistics")
using Statistics

   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [10745b16] + Statistics v1.11.1
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


## Loop over several saved models

In [ ]:
using ACEpotentials
using CSV, DataFrames
using JuLIP: Atoms, energy, forces
using Statistics

# -----------------------------
# SETTINGS
# -----------------------------
param_file = "saved models/saved models_W_Tmix_BCC/resultsmix_r4.5.csv"
out_file   = "saved models/saved models_W_Tmix_BCC/timings.csv"        # results CSV
model_dir  = "saved models/saved models_W_Tmix_BCC"

raw_data = train_data_new_format   # a vector of AtomsData

# -----------------------------
# READ PARAMETER FILE
# -----------------------------
params = CSV.read(param_file, DataFrame)

# -----------------------------
# RESULTS CONTAINER
# -----------------------------
results = DataFrame(
    order = Int[],
    totaldegree = Int[],
    rcut = Float64[],
    mean_force_ms = Float64[],
    median_force_ms = Float64[],
    std_force_ms = Float64[],
    mean_energy_ms = Float64[],
    median_energy_ms = Float64[],
    std_energy_ms = Float64[]
)

# -----------------------------
# LOOP OVER MODELS
# -----------------------------
for row in eachrow(params)
    order = row.order
    totaldegree = row.totaldegree
    rcut = row.rcut

    model_path = joinpath(model_dir,
        "W_Dataset_Tmix_order$(order)_totaldegree$(totaldegree)_rcut$(rcut).json")

    println("Loading model: $model_path")
    model_rl = ACEpotentials.load_potential(model_path, new_format=false)

    # --- Collect per-structure timings ---
    energy_times = Float64[]
    force_times  = Float64[]

    for d in raw_data
        atoms = d.atoms

        # Energy timing
        t1 = time()
        _ = energy(model_rl, atoms)
        t2 = time()
        push!(energy_times, (t2 - t1) * 1000)   # ms

        # Force timing
        t1 = time()
        _ = forces(model_rl, atoms)
        t2 = time()
        push!(force_times, (t2 - t1) * 1000)    # ms
    end

    mean_force   = mean(force_times)
    median_force = median(force_times)
    std_force    = std(force_times)

    mean_energy   = mean(energy_times)
    median_energy = median(energy_times)
    std_energy    = std(energy_times)

    println(" Force [ms]:  mean=$(round(mean_force, digits=3)), median=$(round(median_force, digits=3)), std=$(round(std_force, digits=3))")
    println(" Energy [ms]: mean=$(round(mean_energy, digits=3)), median=$(round(median_energy, digits=3)), std=$(round(std_energy, digits=3))")

    push!(results, (order, totaldegree, rcut,
        mean_force, median_force, std_force,
        mean_energy, median_energy, std_energy))
end

# -----------------------------
# SAVE TO CSV
# -----------------------------
CSV.write(out_file, results)
println("Saved timings to $out_file")


Loading model: /content/drive/MyDrive/Colab_files/Thesis/ACE/fit ACE model/saved models/saved models_W_Tmix_BCC/W_Dataset_Tmix_order3_totaldegree3_rcut4.5.json

This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.5
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.5
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

 Force [ms]:  mean=2.253, median=1.234, std=1.696
 Energy [ms]: mean=2.146, median=1.112, std=3.289
Loading model: /content/drive/MyDrive/Colab_files/Thesis/ACE/fit ACE model/saved models/saved models_W_Tmix_BCC/W_Dataset_Tmix_order3_totaldegree3_rcut7.5.json

This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.5
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.5
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

 Force [ms]:  mean=7.049, median=6.1, std=3.038
 Energy [ms]: mean=4.62, median=3.95

## Single model timing

In [ ]:
using ACEpotentials
using CSV, DataFrames
using JuLIP: Atoms, energy, forces
using Statistics



order = 4
totaldegree = 9
rcut =4.5
# -----------------------------
# SETTINGS
# -----------------------------
model_dir  = "saved models/saved models_W_Tmix_BCC/LBR solver"

raw_data = train_data_new_format   # a vector of AtomsData

# -----------------------------
# LOAD MODEL
# -----------------------------

model_path = joinpath(model_dir,
    "W_Dataset_Tmix_order$(order)_totaldegree$(totaldegree)_rcut$(rcut).json")

println("Loading model: $model_path")
model_rl = ACEpotentials.load_potential(model_path, new_format=false)

# --- Collect per-structure timings (normalized per atom) ---
energy_times = Float64[]
force_times  = Float64[]
for j in 1:10
    for d in raw_data
        atoms = d.atoms
        N = length(atoms)

        # Energy timing
        t1 = time()
        _ = energy(model_rl, atoms)
        t2 = time()
        push!(energy_times, ((t2 - t1) * 1000) / N)   # ms per atom

        # Force timing
        t1 = time()
        _ = forces(model_rl, atoms)
        t2 = time()
        push!(force_times, ((t2 - t1) * 1000) / N)    # ms per atom
    end
end
mean_force   = mean(force_times)
median_force = median(force_times)
std_force    = std(force_times)

mean_energy   = mean(energy_times)
median_energy = median(energy_times)
std_energy    = std(energy_times)

println(" Force [ms/atom]:  mean=$(round(mean_force, digits=4)), median=$(round(median_force, digits=4)), std=$(round(std_force, digits=4))")
println(" Energy [ms/atom]: mean=$(round(mean_energy, digits=4)), median=$(round(median_energy, digits=4)), std=$(round(std_energy, digits=4))")




Loading model: /content/drive/MyDrive/Colab_files/Thesis/ACE/fit ACE model/saved models/saved models_W_Tmix_BCC/LBR solver/W_Dataset_Tmix_order4_totaldegree9_rcut4.5.json

This potential was saved with following versions:

JuLIP v0.14.8
ACEbase v0.4.5
ACE1x v0.1.11
ACE1 v0.12.5
ACEmd v0.1.17
ACEpotentials v0.6.5
ACEfit v0.1.5

If you have problems with using this potential, pin your installation to above versions.

 Force [ms/atom]:  mean=0.0835, median=0.0688, std=0.0704
 Energy [ms/atom]: mean=0.0399, median=0.0301, std=0.054


# Molybdenum (Mo) Dtat and fit model

## Load and split dataset

In [ ]:
path_to_xyz_file = "Mo_random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250.xyz"

"/content/drive/MyDrive/Colab_files/Thesis/MACE/Generate Synthetic structures MACE/Curvature Snapshots/Mo_random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250.xyz"

In [ ]:
mo_data_xyz = read_extxyz(path_to_xyz_file)
length(mo_data_xyz)

176

In [ ]:
println(mo_data_xyz[3].data)

Dict{Any, JuLIP.JData{Float64}}("energy_per_atom" => JuLIP.JData{Float64}(Inf, 0.0, -10.44579766117498), "forces" => JuLIP.JData{Float64}(Inf, 0.0, StaticArraysCore.SVector{3, Float64}[[0.0372479, 0.141572, -0.12500411], [-0.03724789, -0.14157191, 0.12500434]]), "sign" => JuLIP.JData{Float64}(Inf, 0.0, 1), "probe_id" => JuLIP.JData{Float64}(Inf, 0.0, 1), "energy" => JuLIP.JData{Float64}(Inf, 0.0, -20.888731002807617), "origin" => JuLIP.JData{Float64}(Inf, 0.0, "W_base0"))


In [ ]:
println(mo_data_xyz[1].data.keys)

Any["energy_per_atom", #undef, "forces", #undef, "sign", #undef, #undef, "probe_id", #undef, #undef, "energy", #undef, #undef, #undef, #undef, "origin"]


In [ ]:
mo_data_train_xyz = mo_data_xyz[1:1:end]
println(mo_data_train_xyz)
println(length(mo_data_train_xyz))


Atoms[JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.

In [ ]:
mo_data_test_xyz = mo_data_xyz[2:2:end]
println(mo_data_test_xyz)
println(length(mo_data_test_xyz))

Atoms[JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.

## Fit single model

In [ ]:
# This controls the models hyperparamets
order = 4
totaldegree = 9
rcut = 4.5
hyperparams = (elements = [:Mo], #Change this line to the element interested in
					order = order,
					totaldegree = totaldegree,
					rcut = rcut,
					Eref = [:Mo => -4.62891797]) # Change this line to the isolated atom value of the foundation model or method # from MACE MP0 L1 = -4.62891797, from DFT  = --, from MACE-MP0b3_A13_MD_ft = -4.60211419,
model = acemodel(; hyperparams...)
@show length(model.basis);


weights = Dict(
        "W" => Dict("E" => 1.0, "F" => 1.0 , "V" => 1.0 ));

# The solver can be changed too

#solver = ACEfit.LSQR()

solver = ACEfit.RRQR()

#solver = ACEfit.BLR();
#ACEfit.BLR(Dict{Any, Any}());
data_keys = (energy_key = "energy", force_key = "forces");

length(model.basis) = 104


In [ ]:
# This is solving for the parameters of the linear model. Test for XYZ file

result = acefit!( model, mo_data_train_xyz, solver=solver, weights=weights; data_keys...);

┌─────────┬──────────┬───────┬─────┬───────┬──────┐
│    Type │ #Configs │ #Envs │  #E │    #F │   #V │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│ default │      176 │ 21196 │ 176 │ 63588 │    0 │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│   total │      176 │ 21196 │ 176 │ 63588 │    0 │
│ missing │        0 │     0 │   0 │     0 │ 1056 │
└─────────┴──────────┴───────┴─────┴───────┴──────┘


[ Info: Assembling linear problem.
[ Info:   - Creating feature matrix with size (63764, 104).
[ Info:   - Beginning assembly with processor count:  1.
Progress: 100%|█████████████████████████████████████████| Time: 0:02:34
[ Info:   - Assembly completed.
[ Info: Assembling full weight vector.


In [ ]:

@info("Training Errors")
train_res = ACEpotentials.linear_errors(mo_data_train_xyz, model; data_keys...);

@info("Test Errors")
test_res = ACEpotentials.linear_errors(mo_data_test_xyz, model; data_keys...);

@info("Validation Errors")
validation_res = ACEpotentials.linear_errors(mo_data_validation_xyz, model; data_keys...);

[ Info: Training Errors


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   4.814 │    0.007 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   4.814 │    0.007 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   2.747 │    0.004 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   2.747 │    0.004 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table
[ Info: Test Errors


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   4.815 │    0.007 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   4.815 │    0.007 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   2.747 │    0.004 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   2.747 │    0.004 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table
[ Info: Validation Errors


LoadError: UndefVarError: `mo_data_validation_xyz` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
println(train_res["rmse"]["set"]["E"]*1000)
println(train_res["rmse"]["set"]["F"])
println(train_res["mae"]["set"]["E"]*1000)
println(train_res["mae"]["set"]["F"])

4.814488371519978
0.0066348164175695275
2.7465106548222415
0.00434036208253423


In [ ]:
ACEpotentials.save_potential("/content/drive/MyDrive/Colab_files/Thesis/ACE/fit ACE model/Mo/saved models/Mo_Dataset_order$(order)_totaldegree$(totaldegree)_rcut$(rcut).json", model)

In [ ]:
import JuLIP: Atoms, energy, forces, mat

train_data_new_format = [ AtomsData(at; energy_key = "energy", force_key="forces",
                              v_ref = model.Vref)
                for at in mo_data_train_xyz ] ;

test_data_new_format = [ AtomsData(at; energy_key = "energy", force_key="forces",
                              v_ref = model.Vref)
                for at in mo_data_test_xyz ] ;

In [ ]:
model_rl = ACEpotentials.load_potential("/Mo/saved models/saved models_W_Tmix_BCC/W_Dataset_Tmix_order4_totaldegree9_rcut5.5.json", new_format=false)
forces1 = forces(model_rl, test_data_new_format[1].atoms)
energy(model_rl, test_data_new_format[1].atoms)

In [ ]:
# Energy
energy(model.potential,train_data_new_format[1].atoms)

In [ ]:
# Force
forces(model.potential,train_data_new_format[1].atoms)

16-element Vector{StaticArraysCore.SVector{3, Float64}}:
 [0.0051153155567308164, 0.16514561178587203, -0.05700382730949348]
 [0.10851138093786962, -0.07777734759775769, -0.0016585480018080374]
 [0.0157546169537701, -0.032136831290742955, 0.03901973412185853]
 [-0.05599134589249224, -0.04481263288436422, 0.032828525968331225]
 [-0.14970098310504135, -0.09421033369977427, -0.0012411599332393247]
 [0.05549649548325392, -0.01761059388946822, 0.07442295250520585]
 [0.007613175324397649, 0.00180285221503851, 0.0579507552937048]
 [0.056342397575147654, -0.021487306780593393, -0.10251784075196027]
 [-0.035719982544129385, 0.0890113196599652, -0.011958045874583703]
 [-0.007215875731800637, 0.029451983943608304, -0.09815953041931313]
 [-0.060121616121291965, 0.06508437462703462, -0.026808289606733374]
 [0.08324546197455297, -0.016067813101430347, 0.0940272460530172]
 [0.07124667038495447, -0.0030024988454031704, 0.0556022157907373]
 [0.009777505499863644, -0.10500589081646697, -0.03138643746813

In [ ]:
# Time test for forces
t1 = time();

N_comp = 1000 # Number of repeats

for i in 1:N_comp
  forces(model_rl,train_data_new_format[1].atoms);
end

elapsed_time = time() - t1;

print("Time per force computation in ms: ")
print(elapsed_time/N_comp * 1000)
print(" ms \n")


t1 = time();
for i in 1:N_comp
  energy(model_rl,train_data_new_format[1].atoms);
end
elapsed_time = time() - t1;

print("Time per energy computation in ms: ")
print(elapsed_time/N_comp * 1000)
print(" ms \n")

# Niobium (Nb) Dtat and fit model

## Load and split dataset

In [ ]:
path_to_xyz_file = "Nb_random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250.xyz"

"/content/drive/MyDrive/Colab_files/Thesis/MACE/Generate Synthetic structures MACE/Curvature Snapshots/Nb_random_disp_aug_of_evwindowbccfcc_m2_delta0.4&of_relaxedvacancy250.xyz"

In [ ]:
using Pkg; Pkg.add("StatsBase")
using StatsBase

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [ ]:
nb_data_xyz = read_extxyz(path_to_xyz_file)
length(nb_data_xyz)

200

In [ ]:
# randomly choose 176 unique frames (no replacement)
idx = sample(1:length(nb_data_xyz), 176; replace=false)
nb_data_train_xyz = nb_data_xyz[idx];
length(nb_data_train_xyz)

176

In [ ]:
println(nb_data_xyz[3].data)

Dict{Any, JuLIP.JData{Float64}}("energy_per_atom" => JuLIP.JData{Float64}(Inf, 0.0, -9.701778295921695), "forces" => JuLIP.JData{Float64}(Inf, 0.0, StaticArraysCore.SVector{3, Float64}[[0.05940655, 0.22653714, -0.19986355], [-0.05940656, -0.22653718, 0.19986357]]), "sign" => JuLIP.JData{Float64}(Inf, 0.0, 1), "probe_id" => JuLIP.JData{Float64}(Inf, 0.0, 1), "energy" => JuLIP.JData{Float64}(Inf, 0.0, -19.398887634277344), "origin" => JuLIP.JData{Float64}(Inf, 0.0, "W_base0"))


In [ ]:
println(nb_data_xyz[1].data.keys)

Any["energy_per_atom", "relax_steps", "forces", #undef, #undef, #undef, #undef, #undef, "id", #undef, "energy", #undef, #undef, #undef, "level", #undef]


In [ ]:
nb_data_train_xyz = nb_data_xyz[1:1:end]
println(nb_data_train_xyz)
println(length(nb_data_train_xyz))


Atoms[JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.

In [ ]:
nb_data_test_xyz = nb_data_xyz[2:2:end]
println(nb_data_test_xyz)
println(length(nb_data_test_xyz))

Atoms[JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 2 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.Atoms with 4 atoms, JuLIP.

## Fit single model

In [ ]:
# This controls the models hyperparamets
order = 4
totaldegree = 9
rcut = 4.5
hyperparams = (elements = [:Nb], #Change this line to the element interested in
					order = order,
					totaldegree = totaldegree,
					rcut = rcut,
					Eref = [:Nb => -3.22328741]) # Change this line to the isolated atom value of the foundation model or method # from MACE MP L1 = -3.22328741, from DFT  = --, from MACE-MP0b3_A13_MD_ft = -3.13429753,
model = acemodel(; hyperparams...)
@show length(model.basis);


weights = Dict(
        "W" => Dict("E" => 1.0, "F" => 1.0 , "V" => 1.0 ));

# The solver can be changed too

#solver = ACEfit.LSQR()

solver = ACEfit.RRQR()

#solver = ACEfit.BLR();
#ACEfit.BLR(Dict{Any, Any}());
data_keys = (energy_key = "energy", force_key = "forces");

length(model.basis) = 104


In [ ]:
# This is solving for the parameters of the linear model. Test for XYZ file

result = acefit!( model, nb_data_train_xyz, solver=solver, weights=weights; data_keys...);

┌─────────┬──────────┬───────┬─────┬───────┬──────┐
│    Type │ #Configs │ #Envs │  #E │    #F │   #V │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│ default │      176 │ 17999 │ 176 │ 53997 │    0 │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│   total │      176 │ 17999 │ 176 │ 53997 │    0 │
│ missing │        0 │     0 │   0 │     0 │ 1056 │
└─────────┴──────────┴───────┴─────┴───────┴──────┘


[ Info: Assembling linear problem.
[ Info:   - Creating feature matrix with size (54173, 104).
[ Info:   - Beginning assembly with processor count:  1.
Progress: 100%|█████████████████████████████████████████| Time: 0:02:38
[ Info:   - Assembly completed.
[ Info: Assembling full weight vector.


In [ ]:

@info("Training Errors")
train_res = ACEpotentials.linear_errors(nb_data_train_xyz, model; data_keys...);

@info("Test Errors")
test_res = ACEpotentials.linear_errors(nb_data_test_xyz, model; data_keys...);

@info("Validation Errors")
validation_res = ACEpotentials.linear_errors(nb_data_validation_xyz, model; data_keys...);

[ Info: Training Errors


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   3.144 │    0.003 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   3.144 │    0.003 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   2.342 │    0.002 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   2.342 │    0.002 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table
[ Info: Test Errors


┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   3.113 │    0.003 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   3.113 │    0.003 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘
┌─────────┬─────────┬──────────┬─────────┐
│    Type │ E [meV] │ F [eV/A] │ V [meV] │
├─────────┼─────────┼──────────┼─────────┤
│ default │   2.293 │    0.002 │   0.000 │
├─────────┼─────────┼──────────┼─────────┤
│     set │   2.293 │    0.002 │   0.000 │
└─────────┴─────────┴──────────┴─────────┘


[ Info: RMSE Table
[ Info: MAE Table
[ Info: Validation Errors


LoadError: UndefVarError: `nb_data_validation_xyz` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [ ]:
println(train_res["rmse"]["set"]["E"]*1000)
println(train_res["rmse"]["set"]["F"])
println(train_res["mae"]["set"]["E"]*1000)
println(train_res["mae"]["set"]["F"])

3.1437530448609765
0.002685038600337626
2.3423665033696692
0.0019219129712816287


In [ ]:
ACEpotentials.save_potential("Nb_Dataset_order$(order)_totaldegree$(totaldegree)_rcut$(rcut).json", model)

In [ ]:
import JuLIP: Atoms, energy, forces, mat

train_data_new_format = [ AtomsData(at; energy_key = "energy", force_key="forces",
                              v_ref = model.Vref)
                for at in mo_data_train_xyz ] ;

test_data_new_format = [ AtomsData(at; energy_key = "energy", force_key="forces",
                              v_ref = model.Vref)
                for at in mo_data_test_xyz ] ;

In [ ]:
model_rl = ACEpotentials.load_potential("/content/drive/MyDrive/Colab_files/Thesis/ACE/fit ACE model/Mo/saved models/saved models_W_Tmix_BCC/W_Dataset_Tmix_order4_totaldegree9_rcut5.5.json", new_format=false)
forces1 = forces(model_rl, test_data_new_format[1].atoms)
energy(model_rl, test_data_new_format[1].atoms)

In [ ]:
# Energy
energy(model.potential,train_data_new_format[1].atoms)

In [ ]:
# Force
forces(model.potential,train_data_new_format[1].atoms)

16-element Vector{StaticArraysCore.SVector{3, Float64}}:
 [0.0051153155567308164, 0.16514561178587203, -0.05700382730949348]
 [0.10851138093786962, -0.07777734759775769, -0.0016585480018080374]
 [0.0157546169537701, -0.032136831290742955, 0.03901973412185853]
 [-0.05599134589249224, -0.04481263288436422, 0.032828525968331225]
 [-0.14970098310504135, -0.09421033369977427, -0.0012411599332393247]
 [0.05549649548325392, -0.01761059388946822, 0.07442295250520585]
 [0.007613175324397649, 0.00180285221503851, 0.0579507552937048]
 [0.056342397575147654, -0.021487306780593393, -0.10251784075196027]
 [-0.035719982544129385, 0.0890113196599652, -0.011958045874583703]
 [-0.007215875731800637, 0.029451983943608304, -0.09815953041931313]
 [-0.060121616121291965, 0.06508437462703462, -0.026808289606733374]
 [0.08324546197455297, -0.016067813101430347, 0.0940272460530172]
 [0.07124667038495447, -0.0030024988454031704, 0.0556022157907373]
 [0.009777505499863644, -0.10500589081646697, -0.03138643746813

In [ ]:
# Time test for forces
t1 = time();

N_comp = 1000 # Number of repeats

for i in 1:N_comp
  forces(model_rl,train_data_new_format[1].atoms);
end

elapsed_time = time() - t1;

print("Time per force computation in ms: ")
print(elapsed_time/N_comp * 1000)
print(" ms \n")


t1 = time();
for i in 1:N_comp
  energy(model_rl,train_data_new_format[1].atoms);
end
elapsed_time = time() - t1;

print("Time per energy computation in ms: ")
print(elapsed_time/N_comp * 1000)
print(" ms \n")